In [17]:
import time
import torch
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
import random
from experiments.pytorch_tablu.utils import preprocess
# import numpy as np
import pandas as pd
# import wandb
from rich import print

from typing import Callable
from datetime import datetime

from time import time
from pytorch_tabular import model_sweep
import warnings

# from pytorch_tabular import TabularModel
# from pytorch_tabular.models import CategoryEmbeddingModelConfig
from pytorch_tabular.config import (
    DataConfig,
    OptimizerConfig,
    TrainerConfig,
    ExperimentConfig,
)
from pytorch_tabular.models.common.heads import LinearHeadConfig


from pytorch_tabular.utils import (
    make_mixed_dataset,
    print_metrics,
    load_covertype_dataset,
)
from pytorch_tabular.utils import get_balanced_sampler, get_class_weighted_cross_entropy

# import os
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:

Feature_Explain = pd.read_excel("data/Amex_ori/Amex Campus.xlsx")
Feature_target_cols = Feature_Explain["Feature Name"][
    Feature_Explain["Extended description"] == "Target"
].to_list()
Feature_cat_col_names = Feature_Explain["Feature Name"][
    Feature_Explain["Feature Type"] == "categorical"
].to_list()
Feature_num_col_names = Feature_Explain["Feature Name"][
    Feature_Explain["Variable Type"] == "numeric"
].to_list()

DEVICE = (
    "mps"
    if torch.backends.mps.is_available()
    else ("gpu" if torch.cuda.is_available() else "cpu")
)
print(DEVICE)

# %%
data = pd.read_parquet("data/Amex_ori/Amex Campus Challenge Train 3.parquet")

data=preprocess(data)

cols = data.columns.to_list()
target_cols = [col for col in cols if col in Feature_target_cols]
if len(target_cols) == 0:
    print("No target column found")
    exit()
# elif len(target_cols) > 1:
#     target_col=target_cols[1]
cat_col_names = [col for col in cols if col in Feature_cat_col_names]
num_col_names = [col for col in cols if col in Feature_num_col_names]

print(f"{target_cols}")
# target_col=cols.unite(Feature_target_cols)

# target_col = 'activation'

# %%

needed_cols = cat_col_names + num_col_names + target_cols
data = data[needed_cols]

data.head()
print(
    f"target_cols={target_cols}, cat_col_names={cat_col_names}, num_col_names={num_col_names}"
)

# %%
# Data: DF, cat_col_names, num_col_names

# 1. activate(0.57%) X recom(12.7%) -> 0~3

# _,data=train_test_split(data,test_size=0.05)




gpu

In [ ]:
data.to_parquet(f"data/train_upsampled_SMOTEEN.parquet")

In [ ]:
train, test = train_test_split(data,test_size=0.2)
# train, val = train_test_split(train,test_size=0.2)

del data
print(f"train.shape={train.shape}, test.shape={test.shape}")


train.shape=(164320, 55), test.shape=(41081, 55)

In [13]:
train.describe()


,merchant_profile_01,customer_spend_01,customer_industry_spend_01,customer_industry_spend_02,customer_industry_spend_03,customer_industry_spend_04,customer_industry_spend_05,customer_spend_02,customer_spend_03,customer_spend_04,...,distance_04,merchant_profile_02,merchant_spend_09,merchant_profile_03,customer_profile_03,customer_digital_activity_02,customer_profile_04,distance_05,ind_recommended,activation
count,164320.000000,164320.000000,164320.000000,164320.000000,1.643200e+05,164320.000000,164320.000000,164320.000000,164320.000000,1.643200e+05,...,164320.000000,164320.000000,164320.000000,164320.000000,164320.000000,164320.000000,164320.000000,164320.000000,164320.000000,164320.000000
mean,182.427412,114.181976,62.442967,28.109490,3.074100e+03,69.250165,51.615345,11.199256,84.204781,7.771210e+04,...,18.110553,0.215232,37460.851917,36604.552774,69.307502,17.764579,165.035289,4.308126,0.677173,0.015744
std,127.141686,281.442665,184.422695,29.262487,1.187780e+04,82.171297,52.385775,6.845946,53.029666,1.896269e+05,...,100.460769,0.102958,30092.053600,29469.246797,25.979744,21.789195,127.777468,4.126736,0.467559,0.124483
min,101.000000,1.000000,1.000000,1.000000,1.000000e+00,1.000000,1.000000,0.000000,0.000000,1.000000e+00,...,1.000000,0.005025,11.000000,1.000000,0.000000,0.000000,7.000000,0.004460,0.000000,0.000000
25%,101.000000,46.692435,24.186259,7.000000,6.060778e+02,15.000000,13.000000,6.000000,44.771175,6.935530e+03,...,1.663346,0.140000,10160.763227,9813.442680,50.847090,5.333333,62.000000,1.642973,0.000000,0.000000
50%,101.000000,68.684122,36.435037,18.000000,1.589288e+03,40.000000,33.155596,10.000000,74.000000,2.284864e+04,...,3.335294,0.202765,28979.334829,28022.000000,75.506529,10.790382,130.000000,3.086960,1.000000,0.000000
75%,307.000000,109.179172,58.963218,39.000000,3.523240e+03,92.802549,73.000000,15.000000,113.729004,7.293200e+04,...,6.043042,0.275396,61951.000000,61031.000000,91.943829,22.782670,237.000000,5.480736,1.000000,0.000000
max,501.000000,42059.627273,15520.718109,341.000000,1.383652e+06,1459.000000,353.000000,147.000000,958.000000,8.949405e+06,...,3748.000000,1.000000,103342.000000,98637.000000,100.000000,1234.500000,762.000000,110.614044,1.000000,1.000000


In [14]:

test[['activation', 'ind_recommended']].describe()


,activation,ind_recommended
count,41081.000000,41081.000000
mean,0.015092,0.677247
std,0.121921,0.467535
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,1.000000
75%,0.000000,1.000000
max,1.000000,1.000000


In [15]:

target_col = 'ind_recommended'

data_config = DataConfig(
    target=[target_col],
    # target should always be a list. Multi-targets are only supported for regression. Multi-Task Classification is not implemented
    continuous_cols=num_col_names,
    categorical_cols=cat_col_names,
    num_workers=0,
)
trainer_config = TrainerConfig(
    auto_lr_find=True,  # Runs the LRFinder to automatically derive a learning rate
    batch_size=1024,
    max_epochs=25,
    early_stopping=None,
    # early_stopping="valid_loss",  # Monitor valid_loss for early stopping
    # early_stopping_mode="min",  # Set the mode as min because for val_loss, lower is better
    # early_stopping_patience=5,  # No. of epochs of degradation training will wait before terminating
    checkpoints="valid_loss",  # Save best checkpoint monitoring val_loss
    checkpoints_path="checkpoints",  # Save the checkpoint in the experiment directory
    # checkpoints_save_top_k=5,
    # progress_bar="simple",
    load_best=True,  # After training, load the best checkpoint
    # accelerator=DEVICE,
    trainer_kwargs=dict(enable_model_summary=False),  # Turning off model summary
)
optimizer_config = OptimizerConfig()

head_config = LinearHeadConfig(
    layers="",  # No additional layer in head, just a mapping layer to output_dim
    dropout=0.1,
    initialization="kaiming",
).__dict__  # Convert to dict to pass to the model config (OmegaConf doesn't accept objects)



In [16]:
from pathlib import Path
def save_sweep_results(sweep_df, save_dir, best_model):
    if not Path(save_dir).exists:
        Path.mkdir(save_dir,parents=True)
    sweep_df.drop(columns=["params", "time_taken", "epochs"])
    sweep_df.to_excel(f"{save_dir}/sweep_result.xlsx", index=False)
    best_model.save_model(f"{save_dir}/best_model.ckpt")


def visualize_sweep_results(sweep_df):
    sweep_df.drop(columns=["params", "time_taken", "epochs"]).style.background_gradient(
        subset=["test_accuracy", "test_f1_score"], cmap="RdYlGn"
    ).background_gradient(subset=["time_taken_per_epoch", "test_loss"], cmap="RdYlGn_r")



In [8]:
%%time
# Filtering out the warnings
# t = time()

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    sweep_df, best_model = model_sweep(
        task="classification",  # One of "classification", "regression"
        train=train,
        test=test,
        data_config=data_config,
        optimizer_config=optimizer_config,
        trainer_config=trainer_config,
        model_list="lite",
        common_model_args=dict(head="LinearHead", head_config=head_config),
        metrics=["accuracy", "f1_score"],
        metrics_params=[{}, {"average": "macro"}],
        metrics_prob_input=[False, True],
        rank_metric=("accuracy", "higher_is_better"),
        progress_bar=True,
        verbose=True,
        suppress_lightning_logger=True,
    )
    



Output()

2024-03-09 01:53:49,197 - {pytorch_tabular:329} - INFO - Training CategoryEmbeddingModel

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

Failed to compute suggestion for learning rate because there are not enough points. Increase the loop iteration limits or the size of your dataset/dataloader.


2024-03-09 02:12:03,689 - {pytorch_tabular:368} - INFO - Finished Training CategoryEmbeddingModel

2024-03-09 02:12:03,690 - {pytorch_tabular:369} - INFO - Results: model: CategoryEmbeddingModel, # Params: 55 T,   
epochs: 25, test_loss: nan, test_accuracy: 0.500071108341217, test_f1_score: 0.3332950174808502, time_taken:       
1094.5190346240997, time_taken_per_epoch: 43.78076138496399

2024-03-09 02:12:03,740 - {pytorch_tabular:329} - INFO - Training GANDALFModel

Finding best initial lr:   0%|          | 0/100 [00:00<?, ?it/s]

AssertionError: 

In [9]:

visualize_sweep_results(sweep_df)
save_sweep_results(sweep_df, f"sweep_results for {target_col}",best_model)


NameError: name 'sweep_df' is not defined